In [1]:
import concurrent.futures
import os

def parallel_reduction(data, operation):
    num_threads = min(len(data), os.cpu_count())
    chunk_size = (len(data) + num_threads - 1) // num_threads

    # Divide data into chunks
    chunks = [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

    # Perform reduction on each chunk in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(operation, chunks))

    # Final reduction on aggregated results from all threads
    return operation(results)

def parallel_min(chunk):
    return min(chunk)

def parallel_max(chunk):
    return max(chunk)

def parallel_sum(chunk):
    return sum(chunk)

def parallel_average(chunk):
    return sum(chunk) / len(chunk)

if __name__ == "__main__":
    data = [1, 5, 3, 7, 9, 2, 4, 6, 8, 10]
    
    # Min operation
    min_value = parallel_reduction(data, parallel_min)
    print("Min:", min_value)

    # Max operation
    max_value = parallel_reduction(data, parallel_max)
    print("Max:", max_value)

    # Sum operation
    sum_value = parallel_reduction(data, parallel_sum)
    print("Sum:", sum_value)

    # Average operation
    avg_value = parallel_reduction(data, parallel_average)
    print("Average:", avg_value)

    

Min: 1
Max: 10
Sum: 55
Average: 5.5


In [2]:
from multiprocessing import Process, Array

class Graph:
    def __init__(self, num_nodes):
        self.adj_list = [[] for _ in range(num_nodes)]

    def add_edge(self, src, dest):
        self.adj_list[src].append(dest)
        self.adj_list[dest].append(src)

def dfs(graph, visited, v, traversal_order):
    if not visited[v]:
        visited[v] = 1
        traversal_order.append(v)
        for adj in graph.adj_list[v]:
            dfs(graph, visited, adj, traversal_order)

def parallel_dfs(graph, start):
    num_nodes = len(graph.adj_list)
    visited = Array('i', [0] * num_nodes)
    traversal_order = []

    # Start parallel DFS traversal from all vertices
    processes = []
    for v in range(num_nodes):
        if v != start:
            p = Process(target=dfs, args=(graph, visited, v, traversal_order))
            processes.append(p)
            p.start()

    # Start DFS traversal from the start vertex
    dfs(graph, visited, start, traversal_order)

    # Wait for all processes to finish
    for p in processes:
        p.join()

    # Print DFS traversal
    print("Parallel DFS from vertex", start, ":", traversal_order)

if __name__ == "__main__":
    num_nodes = 6
    graph = Graph(num_nodes)

    # Add edges
    graph.add_edge(5, 1)
    graph.add_edge(5, 3)
    graph.add_edge(1, 2)
    graph.add_edge(1, 4)
    graph.add_edge(3, 4)
    graph.add_edge(2, 0)
    graph.add_edge(4, 0)

    start_vertex = int(input("Enter the start vertex: "))  # Take input for the starting vertex

    # Perform parallel DFS
    parallel_dfs(graph, start_vertex)


Enter the start vertex: 5
Parallel DFS from vertex 5 : [5, 1, 2, 0, 4, 3]


In [3]:
from queue import Queue

class Graph:
    def __init__(self, num_nodes):
        self.adj_list = [[] for _ in range(num_nodes)]

    def add_edge(self, src, dest):
        self.adj_list[src].append(dest)
        self.adj_list[dest].append(src)

def bfs(graph, start):
    num_nodes = len(graph.adj_list)
    visited = [False] * num_nodes
    traversal_order = []

    queue = Queue()
    queue.put(start)
    visited[start] = True

    while not queue.empty():
        v = queue.get()
        traversal_order.append(v)
        for adj in graph.adj_list[v]:
            if not visited[adj]:
                visited[adj] = True
                queue.put(adj)

    return traversal_order

if __name__ == "__main__":
    num_nodes = 6
    graph = Graph(num_nodes)

    # Add edges
    graph.add_edge(5, 1)
    graph.add_edge(5, 3)
    graph.add_edge(1, 2)
    graph.add_edge(1, 4)
    graph.add_edge(3, 4)
    graph.add_edge(2, 0)
    graph.add_edge(4, 0)

    start_vertex = int(input("Enter the start vertex: "))  # Take input for the starting vertex

    # Perform BFS
    traversal_order = bfs(graph, start_vertex)
    print("BFS traversal from vertex", start_vertex, ":", traversal_order)


Enter the start vertex: 5
BFS traversal from vertex 5 : [5, 1, 3, 2, 4, 0]


In [6]:
import multiprocessing

def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]

def parallel_bubble_sort(arr):
    # Split the array into chunks for parallel processing
    num_chunks = multiprocessing.cpu_count()
    chunk_size = max(1, len(arr) // num_chunks)  # Ensure chunk size is at least 1
    chunks = [arr[i:i + chunk_size] for i in range(0, len(arr), chunk_size)]

    # Create processes for each chunk
    processes = []
    for chunk in chunks:
        process = multiprocessing.Process(target=bubble_sort, args=(chunk,))
        process.start()
        processes.append(process)

    # Wait for all processes to finish
    for process in processes:
        process.join()

    # Merge sorted chunks
    sorted_arr = merge_chunks(chunks)
    return sorted_arr

def merge_chunks(chunks):
    sorted_arr = []
    while any(chunks):
        min_val = float('inf')
        min_index = -1
        for i, chunk in enumerate(chunks):
            if chunk and chunk[0] < min_val:
                min_val = chunk[0]
                min_index = i
        sorted_arr.append(chunks[min_index].pop(0))
    return sorted_arr

if __name__ == "__main__":
    arr = [6, 4, 25, 1, 2, 11, 9]
    print("Original array:", arr)
    sorted_arr = parallel_bubble_sort(arr)
    print("Sorted array bubble:", sorted_arr)


Original array: [6, 4, 25, 1, 2, 11, 9]
Sorted array bubble: [1, 2, 4, 6, 9, 11, 25]


In [7]:
import concurrent.futures
import multiprocessing

def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort the left and right halves
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the sorted halves
    return merge(left_half, right_half)

def merge(left, right):
    merged = []
    left_index = right_index = 0

    # Merge the left and right subarrays
    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # Append remaining elements from left and right subarrays
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged

def parallel_merge_sort(arr):
    # Split the array into chunks for parallel processing
    num_chunks = multiprocessing.cpu_count()
    chunk_size = max(1, len(arr) // num_chunks)  # Ensure chunk size is at least 1
    chunks = [arr[i:i + chunk_size] for i in range(0, len(arr), chunk_size)]

    # Use ThreadPoolExecutor to perform parallel merge sort
    with concurrent.futures.ThreadPoolExecutor() as executor:
        sorted_chunks = list(executor.map(merge_sort, chunks))  # Convert generator to list

    # Merge the sorted chunks sequentially
    sorted_arr = merge_chunks(sorted_chunks)
    return sorted_arr

def merge_chunks(chunks):
    sorted_arr = chunks[0]
    for chunk in chunks[1:]:
        sorted_arr = merge(sorted_arr, chunk)
    return sorted_arr

if __name__ == "__main__":
    arr = [640, 34, 2555, 12, 22, 11, 90]
    print("Original array:", arr)
    sorted_arr = parallel_merge_sort(arr)
    print("Sorted array merge:", sorted_arr)


Original array: [640, 34, 2555, 12, 22, 11, 90]
Sorted array merge: [11, 12, 22, 34, 90, 640, 2555]
